## Pirple Exam

    3. The notebook formulas that can analyze the spam news dataset(s) and learn from them.


Requirements

In [ ]:
!pip install spacy==2.2.3 
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install texthero

Modules

In [88]:
import numpy as np
import pandas as pd
import nltk
import texthero as hero
import gensim
import re
import timeit
from dataclasses import dataclass

from typing import List
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [89]:
data = pd.read_csv('./learning_data/ml_learning_data.csv')

In [90]:
data

,description,news_is
0,trump tax cut plan gains momentum after u.s. b...,1
1,whoa! college snowflake freaks out: screams fo...,0
2,russia's lavrov says hopes syrian congress to ...,1
3,cftc poised to lose sole democratic commission...,1
4,trump just slammed u.s. shut to far more than...,0
...,...,...
44893,qatari emir to attend gulf summit despite row:...,1
44894,u.s. academic groups oppose trump's visa-vetti...,1
44895,nbc management “protected the sh*t out of him”...,0
44896,boiler room ep #85 – the return of the social ...,0


In [91]:
data.iloc[:5].style.background_gradient(cmap='GnBu')

Data Transformations Functions

In [92]:
def clean_strings(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    :param dataframe:
    :returns:
    """
    dataframe = hero.clean(data['description'])
    return dataframe

In [93]:
def create_word2vec_model(x_list: pd.Series, DIM=100) -> list:
    """
    :param x:
    :param DIM:
    :returns:
    """
    w2v_model = gensim.models.Word2Vec(
        sentences=x_list, size=DIM,
        window=10, min_count=1)
    
    print(f"""
    Total Vocabularies created:
    {len(w2v_model.wv.vocab)}""")
    
    return w2v_model

In [94]:
def tokenize_description(x: List[str]) -> List[int] and Tokenizer:
    """
    :param x:
    :returns:
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    X = tokenizer.texts_to_sequences(x)
    tk_model = tokenizer
    
    return X, tk_model

In [95]:
def padding_sequence(x, maxlen=1000):
    X = pad_sequences(x, maxlen=maxlen)
    return X

Classification Model

In [96]:
@dataclass
class NewsModel:

    DIM = 100
    maxlen = 1000
    vocab_size = None
    vocab = None
    weight_matrix = None
    model = None


    def get_weight_matrix(self, word_vec_model, tk_model):
        """
        :param model: word2vec model
        """
        # create vocabulary from token
        self.vocab = tk_model
        self.vocab_size = len(self.vocab) + 1

        weight_mat = np.zeros((self.vocab_size, self.DIM))

        for word, idx in self.vocab.items():
            weight_mat[idx] = word_vec_model.wv[word]
        
        self.weight_matrix = weight_mat
        print("Weigh Matrix Created")


    def create_model(self):

        # create sequential model
        model = Sequential()

        # add embedding weights
        model.add(Embedding(
            self.vocab_size,
            output_dim=self.DIM,
            weights=[self.weight_matrix],
            input_length=self.maxlen, trainable=False))
        
        model.add(LSTM(units=128))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy',
            metrics=['acc'])
        
        self.model = model

Execution

In [98]:
# seperate our class variable
y = data['news_is'].values

In [99]:
%timeit

# clean x_features
data_clean_description = clean_strings(data)
X = data_clean_description

In [100]:
# split x features
X = [text.split() for text in X.tolist()]

In [101]:
%timeit
# create word2vec model
w2v_model = create_word2vec_model(X)


    Total Vocabularies created:
    138659


In [102]:
# Tokenize features
X, tk_model = tokenize_description(X)

In [103]:
# padding sequence
X = padding_sequence(X)

### Create NewsModel Object

In [104]:
# bloomberg news model object
bloomberg = NewsModel()

In [105]:
bloomberg.get_weight_matrix(
    w2v_model,
    tk_model.word_index)

Weigh Matrix Created


In [106]:
%%timeit
bloomberg.create_model()

1 loop, best of 5: 529 ms per loop


### Model Summary

In [107]:
bloomberg.model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 100)         13865800  
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 13,983,177
Trainable params: 117,377
Non-trainable params: 13,865,800
_________________________________________________________________


### Train Test Split


In [108]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=25)

In [109]:
bloomberg.model.fit(
    X_train, y_train, validation_split=0.3,
    epochs=1)

737/737 [==============================] - 852s 1s/step - loss: 0.0803 - acc: 0.9734 - val_loss: 0.0425 - val_acc: 0.9869


### Save Model

In [110]:
bloomberg.model.save('./tf_saved/model/bloomberg_news_model.h5')

### Load Model Test

In [111]:
bloomberg_model = tf.keras.models.load_model('./tf_saved/model/bloomberg_news_model.h5')

In [112]:
bloomberg_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 100)         13865800  
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 13,983,177
Trainable params: 117,377
Non-trainable params: 13,865,800
_________________________________________________________________


### Prediction Score

In [113]:
y_pred = (bloomberg_model.predict(X_test) >= 0.5).astype(int)

In [114]:
accuracy_score(y_test, y_pred)

0.9851224944320712